In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
# import geoplot as gplt
from shapely.geometry import Point
import shapely
import reverse_geocoder as rg

import sklearn as sk
import matplotlib as mpl
import matplotlib.pylab as plt
import matplotlib.font_manager as fm
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set(rc={'figure.figsize':(13.7,10.27)})
sns.set_style("whitegrid")
sns.set_color_codes()

In [2]:
from dask.distributed import Client
import dask.bag as db
import dask.dataframe as dd
import dask.array as da
import dask

from ast import literal_eval
from collections.abc import MutableMapping
from collections import Counter
# import h5py
import io
import os

In [3]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection.split import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
import numpy as np
import os, io
from surprise import KNNBasic, KNNWithMeans
from surprise import SVDpp
from surprise import SVD
from surprise import accuracy

In [4]:
Client()

Client Scheduler: tcp://127.0.0.1:59696 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [5]:
joined_df = dd.read_csv('data/joined_df.csv').compute()

In [6]:
joined_df.columns

Index(['Unnamed: 0', 'rating', 'reviewerName', 'categories', 'gPlusPlaceId',
       'gPlusUserId', 'user_lat', 'user_long', 'placeName', 'price', 'address',
       'place_lat', 'place_long'],
      dtype='object')

In [7]:
joined_df = joined_df.loc[:, ~joined_df.columns.str.match('Unnamed')]

In [8]:
joined_df.describe()

,rating,user_lat,user_long,place_lat,place_long
count,4.102647e+06,1.019908e+06,1.019908e+06,4.102647e+06,4.102647e+06
mean,3.963454e+00,3.543534e+01,-4.230559e+01,3.537519e+01,-3.944279e+01
std,1.130273e+00,1.846955e+01,7.340014e+01,1.788234e+01,7.371885e+01
min,0.000000e+00,-9.000000e+01,-1.764769e+02,-5.484173e+01,-1.800000e+02
25%,3.000000e+00,3.305811e+01,-9.571289e+01,3.271132e+01,-9.309486e+01
50%,4.000000e+00,3.975509e+01,-7.516379e+01,3.974303e+01,-7.401006e+01
75%,5.000000e+00,4.530182e+01,6.123373e+00,4.498173e+01,7.134800e+00
max,5.000000e+00,9.000000e+01,1.790000e+02,7.653802e+01,1.800000e+02


In [9]:
joined_df

,rating,reviewerName,categories,gPlusPlaceId,gPlusUserId,user_lat,user_long,placeName,price,address,place_lat,place_long
0,4.0,william spindler,"['Asian Restaurant', 'Chinese Restaurant']",106591714648856494903,100000032416892623125,NaN,NaN,Peking Chinese Restaurant,$$$,"['860 Main St', 'Red Bluff, CA 96080']",40.179159,-122.236162
1,5.0,william spindler,"['European Restaurant', 'Italian Restaurant', ...",109420033090810328045,100000032416892623125,NaN,NaN,Firehouse Pizza,$$,"['734 Main St', 'Red Bluff, CA 96080']",40.178074,-122.235234
2,5.0,william spindler,['Barbecue Restaurant'],111623070919810985923,100000032416892623125,NaN,NaN,Two Buds Barbeque,NaN,"['22825 Antelope Blvd', 'Red Bluff, CA 96080']",40.184955,-122.186557
3,4.0,william spindler,['Restaurant'],113854191152597312098,100000032416892623125,NaN,NaN,Bud's Jolly Kone,NaN,"['455 Antelope Blvd', 'Red Bluff, CA 96080']",40.185961,-122.209742
4,5.0,william spindler,['Mexican Restaurant'],115827996910815192564,100000032416892623125,NaN,NaN,La Corona,$$,"['914 Walnut St', 'Red Bluff, CA 96080']",40.175064,-122.242574
...,...,...,...,...,...,...,...,...,...,...,...,...
196648,2.0,charles mckinney,"['Hamburger Restaurant', 'Fast Food Restaurant']",116458473784504954830,118446742455312620560,NaN,NaN,Fatburger,NaN,"['6780 Cherry Ave', 'Long Beach, CA 90805']",33.878048,-118.168365
196649,2.0,charles mckinney,"['Mexican Restaurant', 'Latin American Restaur...",117332598175065149705,118446742455312620560,NaN,NaN,Super Mex,NaN,"['5660 Atlantic Ave', 'Long Beach, CA 90805']",33.859250,-118.184753
196650,4.0,charles mckinney,"['Hot Dog Restaurant', 'Takeout Restaurant', '...",117868066122653879601,118446742455312620560,NaN,NaN,Wienerschnitzel,$$$,"['1300 E Rosecrans Ave', 'Compton, CA 90221']",33.903287,-118.209676
196651,1.0,charles mckinney,"['Buffet Restaurant', 'American Restaurant']",117952004983617019485,118446742455312620560,NaN,NaN,HomeTown Buffet,$$,"['3102 E Imperial Hwy', 'Lynwood, CA 90262']",33.930282,-118.216058


In [10]:
def places_filter(df, lat_l, lat_h, lon_l, lon_h):
    filtered_places = []
    
    for d in df.itertuples(index=False):
#         print(type(d))
#         print(d.place_lat)
        if ((d.place_lat >= lat_l) \
          & (d.place_lat <= lat_h) \
         & (d.place_long >= lon_l) \
        & (d.place_long <= lon_h)):
            filtered_places.append(d)
    return filtered_places

In [11]:
%%time
usa_df = places_filter(joined_df, 19.50139, 64.85694, -161.75583, -68.01197)

CPU times: user 14.5 s, sys: 492 ms, total: 15 s
Wall time: 15 s


In [12]:
usa_df = pd.DataFrame(usa_df)

In [13]:
usa_df

,rating,reviewerName,categories,gPlusPlaceId,gPlusUserId,user_lat,user_long,placeName,price,address,place_lat,place_long
0,4.0,william spindler,"['Asian Restaurant', 'Chinese Restaurant']",106591714648856494903,100000032416892623125,NaN,NaN,Peking Chinese Restaurant,$$$,"['860 Main St', 'Red Bluff, CA 96080']",40.179159,-122.236162
1,5.0,william spindler,"['European Restaurant', 'Italian Restaurant', ...",109420033090810328045,100000032416892623125,NaN,NaN,Firehouse Pizza,$$,"['734 Main St', 'Red Bluff, CA 96080']",40.178074,-122.235234
2,5.0,william spindler,['Barbecue Restaurant'],111623070919810985923,100000032416892623125,NaN,NaN,Two Buds Barbeque,NaN,"['22825 Antelope Blvd', 'Red Bluff, CA 96080']",40.184955,-122.186557
3,4.0,william spindler,['Restaurant'],113854191152597312098,100000032416892623125,NaN,NaN,Bud's Jolly Kone,NaN,"['455 Antelope Blvd', 'Red Bluff, CA 96080']",40.185961,-122.209742
4,5.0,william spindler,['Mexican Restaurant'],115827996910815192564,100000032416892623125,NaN,NaN,La Corona,$$,"['914 Walnut St', 'Red Bluff, CA 96080']",40.175064,-122.242574
...,...,...,...,...,...,...,...,...,...,...,...,...
2285752,2.0,charles mckinney,"['Hamburger Restaurant', 'Fast Food Restaurant']",116458473784504954830,118446742455312620560,NaN,NaN,Fatburger,NaN,"['6780 Cherry Ave', 'Long Beach, CA 90805']",33.878048,-118.168365
2285753,2.0,charles mckinney,"['Mexican Restaurant', 'Latin American Restaur...",117332598175065149705,118446742455312620560,NaN,NaN,Super Mex,NaN,"['5660 Atlantic Ave', 'Long Beach, CA 90805']",33.859250,-118.184753
2285754,4.0,charles mckinney,"['Hot Dog Restaurant', 'Takeout Restaurant', '...",117868066122653879601,118446742455312620560,NaN,NaN,Wienerschnitzel,$$$,"['1300 E Rosecrans Ave', 'Compton, CA 90221']",33.903287,-118.209676
2285755,1.0,charles mckinney,"['Buffet Restaurant', 'American Restaurant']",117952004983617019485,118446742455312620560,NaN,NaN,HomeTown Buffet,$$,"['3102 E Imperial Hwy', 'Lynwood, CA 90262']",33.930282,-118.216058


# Starting Collaborative filtering RecSys

### Reading File

In [14]:
reader = Reader(rating_scale=(1,5))  #invoke reader instance of surprise library
data=Dataset.load_from_df(usa_df[['gPlusUserId','gPlusPlaceId','rating']], reader)

### Creating holdout set

In [15]:
#create training set
trainingSet, testSet = train_test_split(data, test_size=0.2, train_size=None, random_state=None, shuffle=True)

In [16]:
%%time
algo=SVD(n_factors=10,reg_all=0.01)
algo.fit(trainingSet)
predictions_svd=algo.test(testSet)

In [17]:
accuracy.rmse(predictions_svd,verbose=True)

RMSE: 1.1032


1.1032025418279905

In [18]:
usa_df.head(10)

,rating,reviewerName,categories,gPlusPlaceId,gPlusUserId,user_lat,user_long,placeName,price,address,place_lat,place_long
0,4.0,william spindler,"['Asian Restaurant', 'Chinese Restaurant']",106591714648856494903,100000032416892623125,NaN,NaN,Peking Chinese Restaurant,$$$,"['860 Main St', 'Red Bluff, CA 96080']",40.179159,-122.236162
1,5.0,william spindler,"['European Restaurant', 'Italian Restaurant', ...",109420033090810328045,100000032416892623125,NaN,NaN,Firehouse Pizza,$$,"['734 Main St', 'Red Bluff, CA 96080']",40.178074,-122.235234
2,5.0,william spindler,['Barbecue Restaurant'],111623070919810985923,100000032416892623125,NaN,NaN,Two Buds Barbeque,NaN,"['22825 Antelope Blvd', 'Red Bluff, CA 96080']",40.184955,-122.186557
3,4.0,william spindler,['Restaurant'],113854191152597312098,100000032416892623125,NaN,NaN,Bud's Jolly Kone,NaN,"['455 Antelope Blvd', 'Red Bluff, CA 96080']",40.185961,-122.209742
4,5.0,william spindler,['Mexican Restaurant'],115827996910815192564,100000032416892623125,NaN,NaN,La Corona,$$,"['914 Walnut St', 'Red Bluff, CA 96080']",40.175064,-122.242574
5,5.0,william spindler,['Chinese Restaurant'],116585428624152564242,100000032416892623125,NaN,NaN,China Doll Chinese Restaurant,NaN,"['182 S Main St', 'Red Bluff, CA 96080']",40.170748,-122.228931
6,2.0,Richard Yocom,['Restaurant'],100073820849130920147,100000053212755369563,NaN,NaN,Hardee's / Red Burrito,$$$,"['134 N Hills St', 'Meridian, MS 39305']",32.413658,-88.677648
7,2.0,Richard Yocom,"['Pizza Restaurant', 'European Restaurant']",102333498482915416504,100000053212755369563,NaN,NaN,Pizza Hut,$$$,"['2199 S Byron Butler Pkwy', 'Perry, FL 32348']",30.094690,-83.580572
8,1.0,Richard Yocom,"['Pizza Restaurant', 'European Restaurant']",102471437282277965376,100000053212755369563,NaN,NaN,Hungry Howie's,$$$,"['15028 US Highway 19 S', 'Thomasville, GA 317...",30.800960,-83.935407
9,2.0,Richard Yocom,"['Restaurant', 'American Restaurant']",103519165841762621376,100000053212755369563,NaN,NaN,Ryan's,$$,"['207 S Frontage Rd', 'Meridian, MS 39301']",32.362456,-88.676926


In [19]:
algo.predict('100000053212755369563','106591714648856494903')

Prediction(uid='100000053212755369563', iid='106591714648856494903', r_ui=None, est=3.4220977776365036, details={'was_impossible': False})

In [23]:
first_user_anti

,rating,reviewerName,categories,gPlusPlaceId,gPlusUserId,user_lat,user_long,placeName,price,address,place_lat,place_long
0,4.0,william spindler,"['Asian Restaurant', 'Chinese Restaurant']",106591714648856494903,100000032416892623125,NaN,NaN,Peking Chinese Restaurant,$$$,"['860 Main St', 'Red Bluff, CA 96080']",40.179159,-122.236162
1,5.0,william spindler,"['European Restaurant', 'Italian Restaurant', ...",109420033090810328045,100000032416892623125,NaN,NaN,Firehouse Pizza,$$,"['734 Main St', 'Red Bluff, CA 96080']",40.178074,-122.235234
2,5.0,william spindler,['Barbecue Restaurant'],111623070919810985923,100000032416892623125,NaN,NaN,Two Buds Barbeque,NaN,"['22825 Antelope Blvd', 'Red Bluff, CA 96080']",40.184955,-122.186557
3,4.0,william spindler,['Restaurant'],113854191152597312098,100000032416892623125,NaN,NaN,Bud's Jolly Kone,NaN,"['455 Antelope Blvd', 'Red Bluff, CA 96080']",40.185961,-122.209742
4,5.0,william spindler,['Mexican Restaurant'],115827996910815192564,100000032416892623125,NaN,NaN,La Corona,$$,"['914 Walnut St', 'Red Bluff, CA 96080']",40.175064,-122.242574
5,5.0,william spindler,['Chinese Restaurant'],116585428624152564242,100000032416892623125,NaN,NaN,China Doll Chinese Restaurant,NaN,"['182 S Main St', 'Red Bluff, CA 96080']",40.170748,-122.228931
6,2.0,Richard Yocom,['Restaurant'],100073820849130920147,100000053212755369563,NaN,NaN,Hardee's / Red Burrito,$$$,"['134 N Hills St', 'Meridian, MS 39305']",32.413658,-88.677648
7,2.0,Richard Yocom,"['Pizza Restaurant', 'European Restaurant']",102333498482915416504,100000053212755369563,NaN,NaN,Pizza Hut,$$$,"['2199 S Byron Butler Pkwy', 'Perry, FL 32348']",30.094690,-83.580572
8,1.0,Richard Yocom,"['Pizza Restaurant', 'European Restaurant']",102471437282277965376,100000053212755369563,NaN,NaN,Hungry Howie's,$$$,"['15028 US Highway 19 S', 'Thomasville, GA 317...",30.800960,-83.935407
9,2.0,Richard Yocom,"['Restaurant', 'American Restaurant']",103519165841762621376,100000053212755369563,NaN,NaN,Ryan's,$$,"['207 S Frontage Rd', 'Meridian, MS 39301']",32.362456,-88.676926


### Find user-item pairs with no ratings



In [20]:
%%time
trainset = data.build_full_trainset()


CPU times: user 6.92 s, sys: 436 ms, total: 7.36 s
Wall time: 7.48 s


In [21]:
%%time
algo.fit(trainset)

CPU times: user 1min 2s, sys: 842 ms, total: 1min 2s
Wall time: 1min 3s


In [ ]:
# %%time
# anti_test_set = trainset.build_anti_testset() 
# predictions = algo.test(anti_test_set)

In [22]:
predictions_svd

[Prediction(uid='116964288929271345658', iid='107565815343535384947', r_ui=5.0, est=4.320748467884034, details={'was_impossible': False}),
 Prediction(uid='115478547432917826696', iid='109151799665350439463', r_ui=5.0, est=4.145192744852595, details={'was_impossible': False}),
 Prediction(uid='112782285739893284350', iid='113504585588282949910', r_ui=5.0, est=3.859370679775778, details={'was_impossible': False}),
 Prediction(uid='114064089751245355381', iid='110283443476697135794', r_ui=4.0, est=4.048239058726746, details={'was_impossible': False}),
 Prediction(uid='102549322583401594597', iid='112994664185135842079', r_ui=5.0, est=4.026765626739224, details={'was_impossible': False}),
 Prediction(uid='117732054076765983530', iid='104943997364109909274', r_ui=2.0, est=3.4946763615325303, details={'was_impossible': False}),
 Prediction(uid='112245226914404641499', iid='114225073773264444552', r_ui=1.0, est=3.72102252070682, details={'was_impossible': False}),
 Prediction(uid='1160682428

In [89]:
from collections import defaultdict

def getRecommendations(userID='100000053212755369563', topN=3):
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions_svd[:1000]: 
            predicted_value = algo.predict(userID,iid)
#             selected_row = usa_df.loc[usa_df['gPlusPlaceId']==iid]
#             category = selected_row.iloc[0].categories
#             place_name = selected_row.iloc[0].placeName
            top_recs[iid].append([predicted_value.est])
     
    output_dict = {}
    for iid, item_ratings in sorted(top_recs.items(), key=lambda item: item[1][0], reverse=True):
        output_dict[iid]=item_ratings
     
    return output_dict 

In [90]:
%%time
getRecommendations()

CPU times: user 24 ms, sys: 5.29 ms, total: 29.3 ms
Wall time: 27.8 ms


{'117724633484873036747': [[4.041179256701367]],
 '106536161216920689890': [[4.027982213012628]],
 '113624144608764508129': [[4.009257248606107]],
 '105482766922738531867': [[4.002899114953272]],
 '116504186279920612836': [[3.975970646214979]],
 '109938846905004437020': [[3.90720922307836]],
 '111049703396321758551': [[3.905123845644501]],
 '108739123308426713653': [[3.8869288873905496]],
 '107016265344828034042': [[3.8701258551167084]],
 '117758553453540777193': [[3.840818973642755]],
 '105022128980663060101': [[3.8374644493235093]],
 '118146191909735041123': [[3.831482789042134]],
 '114515529427375734246': [[3.8272181235926674]],
 '105200119434117168158': [[3.8265936468941093]],
 '103440262735799005316': [[3.8259446532576433]],
 '109846408578659628323': [[3.804278345847007]],
 '113286511279599426691': [[3.802284714050939]],
 '111167123137633455783': [[3.7962950776483018]],
 '105601062618239474770': [[3.7928376766064895]],
 '108088757533332872889': [[3.7911118281374607]],
 '1114021152

In [138]:
from math import sin, cos, sqrt, atan2, radians
def calculate_distance(lat1, lon1, lat2, lon2):

    # approximate radius of earth in km
    R = 6373.0

    lat1_r = radians(lat1)
    lon1_r = radians(lon1)
    lat2_r = radians(lat1)
    lon2_r = radians(lon2)

    dlon = lon2_r - lon1_r
    dlat = lat2_r - lat1_r

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [141]:
def getRecommendations_from_df(df, userID='100000053212755369563', topN=3, lat=None, lon=None, distance_limit=100):
    top_recs = defaultdict(list)
    output_dict = {}
    dis = None
    for row in df.itertuples():
        iid = row.gPlusPlaceId
        predicted_value = algo.predict(userID, iid)
        
        if lat and lon:
            try:
                dis = calculate_distance(row.place_lat, row.place_long, lat, lon)
            except Exception as e:
                  pass
#                 print("Exception caught: {}".format(e))
        if dis <= distance_limit:
            top_recs[iid].append({'PlaceName':row.placeName, 'Distance':dis, 'Category':row.categories, 'Prediction':predicted_value.est})
     
    for iid, item_ratings in sorted(top_recs.items(), key=lambda item: (item[1][0].get('Prediction'), item[1][0].get('Distance')), reverse=True):
        output_dict[iid]=item_ratings
     
    return output_dict 

In [155]:
%%time
recs= getRecommendations_from_df(usa_df[:10000], userID='118446742455312620560', lat=40.179159, lon=-122.236162, distance_limit= 0.5)
print(len(recs.items()))
recs

5
CPU times: user 194 ms, sys: 24.6 ms, total: 219 ms
Wall time: 208 ms


{'110040354606321397134': [{'PlaceName': 'New Asian Restaurant',
   'Distance': 0.1560081344773754,
   'Category': "['Asian Restaurant']",
   'Prediction': 3.3727068848729784}],
 '103770440322838225806': [{'PlaceName': 'Franz Bakery Outlet Store',
   'Distance': 0.27501351859849166,
   'Category': "['Bakery']",
   'Prediction': 3.278549165227188}],
 '109420033090810328045': [{'PlaceName': 'Firehouse Pizza',
   'Distance': 0.08141167410460685,
   'Category': "['European Restaurant', 'Italian Restaurant', 'Pizza Restaurant']",
   'Prediction': 3.092806171158202}],
 '106591714648856494903': [{'PlaceName': 'Peking Chinese Restaurant',
   'Distance': 2.2331361724487817e-12,
   'Category': "['Asian Restaurant', 'Chinese Restaurant']",
   'Prediction': 3.0543916521404526}],
 '101530031206675973002': [{'PlaceName': 'Kenmore Lanes',
   'Distance': 0.13456006148815877,
   'Category': "['Bowling Alley', 'Lounge', 'Restaurant']",
   'Prediction': 2.663051763369901}]}